<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install virtualenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 42.9 MB/s eta 0:00:00


In [2]:
!virtualenv rag_services_demo


created virtual environment CPython3.10.12.final.0-64 in 1102ms
  creator CPython3Posix(dest=/content/rag_services_demo, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
!source rag_services_demo/bin/activate

In [4]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [6]:
!pip install -q git+https://github.com/deepset-ai/haystack.git@openapi_container_v5

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.5 MB/s eta 0:00:00


In [7]:
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00


In [8]:
!pip install openapi3

In [9]:
!pip install jsonref

In [10]:
!pip install -q git+https://github.com/qdrant/qdrant-haystack.git@haystack-v2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/4

In [11]:
import getpass
import os
import json
import requests
from qdrant_haystack import QdrantDocumentStore
from qdrant_haystack.retriever import QdrantRetriever

from haystack import Pipeline
from haystack.components.generators.utils import default_streaming_callback
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import GPTChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage

In [12]:
document_store = QdrantDocumentStore(
    path="./qdrant_v1",
    index="Document",
    embedding_dim=768,
    recreate_index=True,
    hnsw_config={"m": 16, "ef_construct": 64}  # Optional
)

In [13]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect(connect_from="spec_to_functions", connect_to="embedder")
indexing_pipeline.connect(connect_from="embedder", connect_to="writer")

In [ ]:
indexing_pipeline.run(data={"sources":["https://t.ly/eBODl", "https://t.ly/Cn-Tv"],
                            "system_messages":[requests.get("https://t.ly/_1vOw").text,
                                               requests.get("https://t.ly/fZR09").text]})

In [15]:
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
retrieval_pipeline.add_component("retriever", QdrantRetriever(document_store=document_store))
retrieval_pipeline.connect(connect_from="embedder", connect_to="retriever")

In [16]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [18]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
services_auth = {"SerperDev":serper_dev_key}

Enter serperdev api key:··········


In [19]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [24]:
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
user_prompt = "Compare branches main and openapi_container_v5, in project deepset-ai, repo haystack"

In [25]:
top_k = 1
top_k_result = retrieval_pipeline.run(data={"text": user_prompt, "top_k": top_k})
top_k_docs = top_k_result["retriever"]["documents"][:top_k]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
top_1_document = top_k_docs[0]
openai_functions_definition = json.loads(top_1_document.content)
openapi_spec = top_1_document.meta["spec"]
messages = [ChatMessage.from_user(user_prompt)]

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"functions": [openai_functions_definition]},
                                                 "service_openapi_spec": openapi_spec})

In [27]:
gen_pipe = Pipeline()
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-4-1106-preview", streaming_callback=default_streaming_callback)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(top_1_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

### Why:
The proposed changes aim to enhance the haystack library by introducing new components for interacting with OpenAPI services. These components extend the functionality of the library to allow it to connect to external services described by OpenAPI specifications, enabling the invocation of those services' methods directly through the framework.

### What:
The pull request consists of the following key additions:

1. `OpenAPIServiceConnector`: This component allows the haystack framework to connect to and interact with services that provide an OpenAPI specification. It enables the execution of service methods based on the content of chat messages that follow a specific JSON structure.

2. `OpenAPIServiceToFunctions`: This converter component processes an OpenAPI specification to extract definitions that can be called via the OpenAI function-calling mechanism.

The changes also include updates to the `__init__.py` files in the `connectors` and `converters` modules to expose the 